In [4]:
import numpy as np
import pandas as pd

# interpretation of return value
# hurst < 0.5 - input_ts is mean reverting
# hurst = 0.5 - input_ts is effectively random/geometric brownian motion
# hurst > 0.5 - input_ts is trending
            
def hurst(input_ts, lags_to_test = 150) :
            tau = []
            lagvec = []  
            #  Step through the different lags  
            for lag in range(2, lags_to_test):  
                #  produce price difference with lag  
                pp = np.subtract(input_ts[lag:], input_ts[:-lag])  
                #  Write the different lags into a vector  
                lagvec.append(lag)  
                #  Calculate the variance of the differnce vector  
                tau.append(np.sqrt(np.std(pp)))  
            #  linear fit to double-log graph (gives power)  
            m = np.polyfit(np.log10(lagvec), np.log10(tau), 1)  
            # calculate hurst  
            hurst = m[0]*2
            return hurst 


stocklist = pd.read_table('stocklist.txt', names = ('A'))
f = open("hurst_scores.csv", "w+")

for stockname in stocklist['A']:
    df_all = pd.read_csv("stock_data/"+stockname+".csv", names=('Dates', 'stockVWAP', 'futureVWAP', 'bidPrice', 'askPrice', 'total_value', 'total_size'))
    df_train = df_all[:60000]
    df_train_resampled_10min = df_train[df_train.index%10 == 0]
    df_train_resampled_10min.reset_index(drop = True)   
    df = df_train_resampled_10min[:5999]

    hurst_score = hurst(df['stockVWAP'])
#     print(stockname+":"+str(hurst(df['stockVWAP'])))
    f.write(stockname+","+str(hurst_score))
    f.write('\n')  

f.close()